# 1. Ensemble CSV 가지고 오기

모든 CSV 파일을 가지고 옵니다.

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/ensemble.zip -d /content/csv_file

Archive:  /content/drive/MyDrive/ensemble.zip
   creating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/
   creating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission2_0.9904_0.0976_soft.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission5_0.9907_0.0921_soft.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission21_0.9990_0.0673_soft.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission16_0.9983_0.0698_hard.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission31_0.9995_0.0661_soft.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission24_0.9991_0.0666_hard.csv  
  inflating: /content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission33_0.9994_0.0658_s

# 2. MASK / GENDER / AGE 끼리 합치기

MASK, GENDER, AGE 각각의 분류에 대한 모델 inference를 ensemble합니다.

In [22]:
import pandas as pd
import numpy as np

In [23]:
submission = pd.read_csv('/content/drive/MyDrive/info.csv')
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0


## 2-1. age

In [51]:
_age_list = [
       '/content/csv_file/age_ecaresnet50t_140_focal_0.01/exp/submission29_0.9947_0.0061_soft.csv',
       '/content/csv_file/age_seresnet152d_64_cross_entropy_0.01/exp/submission34_0.9955_0.0188_soft.csv',

]
age = []

for idx, path in enumerate(_age_list):
    age.append(pd.read_csv(path))

_0 = 0
_1 = 0
_2 = 0

for idx in range(len(_age_list)):
    _0 += age[idx]['0']
    _1 += age[idx]['1']
    _2 += age[idx]['2']


ensemble_age = pd.DataFrame({'ImageID':age[0]['ImageID'],
                             '0':_0/len(_age_list),
                             '1':_1/len(_age_list),
                             '2':_2/len(_age_list),

})
ensemble_age['ans'] = ensemble_age[['0', '1', '2']].idxmax(axis=1)
ensemble_age

,ImageID,0,1,2,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,-6.606654,2.520211,-2.801476,1
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,-9.447104,0.020001,2.755419,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,-8.356492,0.741875,1.821718,2
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,-10.114423,0.873680,3.724512,2
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,6.235811,-2.113149,-10.196953,0
...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,-6.593726,2.248388,-0.824568,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,-7.133375,1.628390,-0.007194,1
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,5.178672,-1.082957,-10.073687,0
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,-2.672913,1.802194,-3.527008,1


In [57]:
ensemble_age['ans'].value_counts()

0    5361
1    5017
2    2222
Name: ans, dtype: int64

## 2-2. gender

In [58]:
_gender_list = [
       '/content/csv_file/gender_seresnet152d_64_cross_entropy_0.01/exp/submission28_0.9995_0.0036_soft.csv',
       '/content/submission14_0.9845_0.0450_soft.csv'
]
gender = []

for idx, path in enumerate(_gender_list):
    gender.append(pd.read_csv(path))


_0 = 0
_1 = 0

for idx in range(len(_gender_list)):
    _0 += gender[idx]['0']
    _1 += gender[idx]['1']


ensemble_gender = pd.DataFrame({'ImageID':gender[0]['ImageID'],
                             '0':_0/len(_gender_list),
                             '1':_1/len(_gender_list),

})
ensemble_gender['ans'] = ensemble_gender[['0', '1']].idxmax(axis=1)
ensemble_gender

,ImageID,0,1,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,3.879424,-3.928607,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,4.071319,-4.132596,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,1.861308,-1.888498,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,3.215397,-3.250312,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,6.820894,-6.893844,0
...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,4.875422,-4.925916,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,-7.338000,7.262609,1
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,-6.242721,6.191085,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0.719521,-0.755907,0


In [59]:
ensemble_gender['ans'].value_counts()

0    7478
1    5122
Name: ans, dtype: int64

## 2-3. mask

In [60]:
_mask_list = [
       '/content/csv_file/mask_Efficientnet_b4_128_cross_entropy_0.01/exp/submission28_0.9998_0.0015_soft.csv',
       '/content/csv_file/mask_Efficientnet_b4_140_f1_0.01/exp/submission33_0.9901_0.0157_soft.csv',
       '/content/csv_file/mask_Efficientnet_b4_140_focal_0.01/exp/submission30_0.9988_0.0012_soft.csv',
       '/content/csv_file/mask_Efficientnet_b6_64_f1_0.01/exp/submission29_0.9763_0.0358_soft.csv',
       '/content/csv_file/mask_ecaresnet50t_128_label_smoothing_0.01/exp/submission33_0.9994_0.0658_soft.csv',
       '/content/csv_file/mask_seresnet152d_16_label_smoothing_0.01/exp/submission29_0.9976_0.0723_soft.csv'
]
mask = []

for idx, path in enumerate(_mask_list):
    mask.append(pd.read_csv(path))

_0 = 0
_1 = 0
_2 = 0

for idx in range(len(_mask_list)):
    _0 += mask[idx]['0']
    _1 += mask[idx]['1']
    _2 += mask[idx]['2']


ensemble_mask = pd.DataFrame({'ImageID':mask[0]['ImageID'],
                             '0':_0/len(_mask_list),
                             '1':_1/len(_mask_list),
                             '2':_2/len(_mask_list),

})
ensemble_mask['ans'] = ensemble_mask[['0', '1', '2']].idxmax(axis=1)
ensemble_mask

,ImageID,0,1,2,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,-5.742853,-2.353159,8.392527,2
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,4.591591,-1.979908,-3.928930,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,-4.021264,-2.638004,6.938831,2
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,-6.513771,-2.090902,8.878636,2
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,-5.951790,-1.839521,8.319632,2
...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0.380004,1.399367,-3.773414,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,5.557627,-2.955921,-3.721393,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,-8.724238,9.998842,-2.624162,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,5.590073,-2.917747,-3.842400,0


In [61]:
ensemble_mask['ans'].value_counts()

0    8908
1    1889
2    1803
Name: ans, dtype: int64

## 2-4. Multi

Inference를 multi label로 했을 경우 바로 ensemble합니다.

In [26]:
MULTI_LABEL_CLASSES = 18

_multi_list = [
       '/content/submission12_0.8927_0.2882_soft.csv',
       '/content/submission14_0.9006_0.2756_soft.csv'
]

multi = []
for idx, path in enumerate(_multi_list):
    multi.append(pd.read_csv(path))


label_names = {}
for idx in range(MULTI_LABEL_CLASSES):
    label_names[str(idx)] = 0

for idx in range(len(_multi_list)):
    for label_idx in range(len(label_names)):
        label_names[str(label_idx)] += multi[idx][str(label_idx)]

label_names = {key : value/len(_multi_list) for key, value in label_names.items()}

ensemble_multi = pd.DataFrame.from_dict(label_names)

ensemble_multi['ans'] = ensemble_multi[[str(idx) for idx in range(MULTI_LABEL_CLASSES)]].idxmax(axis=1)

ensemble_multi.insert(0, 'ImageID', multi[0]['ImageID'])
ensemble_multi

,ImageID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,-8.789786,1.044467,1.352743,-12.561731,-8.176428,-6.656835,-5.235981,3.644729,4.231629,-9.831962,-1.718419,-1.256970,3.093288,10.775663,11.725627,-1.687888,5.216757,5.965401,14
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,-0.596633,6.737746,7.268932,-6.879649,1.390544,2.473943,-5.217544,0.586165,0.908842,-11.449597,-4.221000,-3.254881,-8.281676,-3.175395,-0.810468,-13.491362,-8.820434,-5.931949,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,-7.365077,0.866840,1.969394,-12.894749,-6.028309,-4.309429,-5.805081,2.477539,3.392555,-12.138758,-1.919928,-0.766047,1.961910,9.105279,10.281622,-4.556166,4.587334,5.825099,14
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,-6.209393,1.283693,0.093067,-10.198955,-7.903470,-8.403557,-3.245356,3.944191,3.155128,-7.998347,-1.869347,-2.942067,3.881448,9.723458,9.472798,-1.296765,3.680716,3.150526,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0.882703,-0.857818,-5.347413,-2.880174,-9.281595,-12.674307,4.025595,0.459629,-3.581845,-0.434107,-5.104191,-9.170486,10.144574,6.346302,1.953159,6.327623,0.399754,-3.193491,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1.804113,7.578448,6.505082,-3.700635,-0.296342,-0.530355,-4.292648,0.355472,-0.688453,-10.439084,-6.393520,-6.719277,-4.948807,-0.549236,0.275461,-10.453388,-7.147765,-5.841287,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,-3.422627,3.608278,1.662317,-3.557315,5.363649,4.009104,-8.528472,-2.704669,-4.271184,-9.030179,-1.806237,-3.001453,-11.887949,-6.565454,-6.511147,-11.785713,-5.979514,-5.418150,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,-4.514195,-6.981222,-11.756635,2.247715,-1.945004,-5.803163,1.254874,-1.248153,-4.248010,7.343738,3.584236,-0.444558,-2.871043,-5.668343,-8.542380,2.539003,-1.157938,-4.008831,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1.371807,5.537806,2.260230,-1.316087,3.863547,0.782548,-4.619415,-1.272525,-4.258722,-7.612881,-3.253829,-6.096030,-8.323011,-5.280622,-6.244581,-10.520010,-7.631996,-8.423676,1


In [27]:
ensemble_multi['ans'].value_counts()

0     2292
2     1889
3     1529
4     1488
1     1286
5      505
14     475
6      457
12     454
8      380
15     323
9      314
16     291
10     272
7      257
13     168
11     122
17      98
Name: ans, dtype: int64

# 3. Ensemble submission

## 3-1. MASK + AGE + GENDER

MASK + AGE + GENDER에 각각의 값을 곱해 최종 label을 완성합니다.

In [62]:
submission['ans'] = pd.to_numeric(ensemble_mask['ans']) *6 + pd.to_numeric(ensemble_gender['ans'])*3 + pd.to_numeric(ensemble_age['ans'])
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,14
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,14
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,7
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [63]:
submission.to_csv('submission.csv', index=False)

## 3-2. Multi

Multi label의 최종 submission을 완성합니다.

In [28]:
submission['ans'] = pd.to_numeric(ensemble_multi['ans'])
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,14
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,14
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [29]:
submission.to_csv('submission.csv', index=False)